In [12]:
import brickschema
from tqdm import tqdm
from smap.archiver.client import SmapClient
from smap.contrib import dtutil
import pandas as pd

In [13]:
g = brickschema.Graph()
g.load_file('2022_sdh_brick_expanded.ttl')

<Graph identifier=Nb6080cbe431143caaf83f3428da14a62 (<class 'brickschema.graph.Graph'>)>

In [14]:
# Query test 1
# query1 = g.query(
#     """SELECT ?CH ?points WHERE {
#     ?CH a brick:Chiller .
#     ?CH brick:hasPoint ?points .
# }"""
# )

# print("\nQuery test 1 results:")
# for row in query1:
#     print(row)

In [24]:
# Query for return temperature (approximated from zone temperature)
zoneT_query = g.query(
    """SELECT DISTINCT ?ahu ?zone ?sensor ?bacnet_addr ?bacnect_instance ?bacnect_id WHERE {
    ?ahu            rdf:type                        brick:AHU .
    ?zone           rdf:type                        brick:HVAC_Zone .
    ?sensor         rdf:type/rdfs:subClassOf*       brick:Zone_Air_Temperature_Sensor .
    ?ahu            brick:feeds                     ?vav .
    ?vav            rdf:type                        brick:VAV .
    ?vav            brick:feeds                     ?zone .
    ?zone           brick:hasPoint                  ?sensor .
    VALUES          ?req_point                      ?sensor .
    ?sensor         brick:bacnetPoint               ?bacnet_id .
    ?bacnet_id      brick:hasBacnetDeviceInstance   ?bacnet_instance .
    ?bacnet_id      brick:hasBacnetDeviceType       ?bacnet_type .
    ?bacnet_id      brick:accessedAt                ?bacnet_net .
    ?bacnet_net     sdh:connstring                  ?bacnet_addr .
}"""
)

print("\nQuery test 1 results:")

for row in tqdm(zoneT_query):
    print(row)


Query test 1 results:


KeyboardInterrupt: 

In [178]:
# Query for zone supply temperature and return temperature
sat_query = g.query(
    """SELECT DISTINCT ?ahu ?sensor ?bacnet_id ?bacnet_instance WHERE {
    ?ahu            rdf:type                        brick:AHU .
    ?sensor         rdf:type/rdfs:subClassOf*       brick:Supply_Air_Temperature_Sensor .
    ?ahu            brick:hasPoint                  ?sensor .
    ?sensor         brick:bacnetPoint               ?bacnet_id .
    ?bacnet_id      brick:hasBacnetDeviceInstance   ?bacnet_instance .
    ?bacnet_id      brick:hasBacnetDeviceType       ?bacnet_type .
    ?bacnet_id      brick:accessedAt                ?bacnet_net .
    ?bacnet_net     sdh:connstring                  ?bacnet_addr .
}"""
)

df = pd.DataFrame(sat_query, columns=[str(s) for s in sat_query.vars])
point_name = []
for i in range(len(df)):
    point_name.append(df.sensor.str.split("/", expand=False)[i][4].split("#")[1])
df['point_name'] = point_name
df["bacnet_instance"] = df["bacnet_instance"].astype(int).astype(str)

In [172]:
def get_paths_from_tags(tags):
    paths = {key: tags[key]["Path"] for key in tags}
    paths = pd.DataFrame.from_dict(paths, orient='index', columns=['path'])
    new_cols = ["empty", "site", "device_number", "point_name", "bacnet_instance", "property_name"]

    # adjustments to dataframe
    paths[new_cols] = paths.path.str.split("/", expand=True)
    paths = paths.drop(columns=["empty"])
    paths['point_name'] = paths['point_name'].str.replace('SDH.', '', regex=False)
    return paths

In [173]:
def get_data_from_smap(points_to_download, paths, smap_client, start, end):
    df_combine = pd.merge(paths.reset_index(), points_to_download, how='inner', on=['bacnet_instance', 'point_name'])
    df_combine = df_combine[df_combine['property_name'] == 'presentValue']

    # combine the data frames

    # get data from smap
    data = smap_client.data_uuid(df_combine["index"], start, end, cache=False)
    return df_combine, data

In [174]:
url = "http://178.128.64.40:8079"
keyStr = "B7qm4nnyPVZXbSfXo14sBZ5laV7YY5vjO19G"
where = "Metadata/SourceName = 'Field Study 5a'"
start = dtutil.dt2ts(dtutil.strptime_tz("2023-01-20", "%Y-%m-%d"))
end = dtutil.dt2ts(dtutil.strptime_tz("2023-01-25", "%Y-%m-%d"))

In [175]:
smap_client = SmapClient(url, key=keyStr)
tags = smap_client.tags(where, asdict=True)
paths = get_paths_from_tags(tags)
points_to_download, data = get_data_from_smap(df, paths, smap_client, start, end)